## Variational Autoencoder with Enhanced Clustering for Health Severity Analysis
This notebook implements an improved Variational Autoencoder (VAE) on structured health data to identify patient clusters corresponding to different levels of health severity. We've incorporated various architecture styles, expanded hyperparameter tuning, and optimized the code for better performance and resource utilization.

#### Library Imports

In [3]:
# Import standard libraries
import numpy as np
import pandas as pd
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import tqdm as notebook_tqdm

# Machine learning and deep learning libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.manifold import TSNE
import umap.umap_ as umap

import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner
from keras_tuner import BayesianOptimization

# Enable inline plotting
%matplotlib inline


#### Check GPU Availability and Configure Memory Growth

In [4]:
# Check if TensorFlow is using the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Enable memory growth for GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Enabled memory growth on GPU")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. The code will run on CPU, which might be slower.")


Num GPUs Available:  1
Enabled memory growth on GPU


#### Load and Preprocess the Enhanced Data

Load Patient Data with Sequences

In [ ]:
# Load patient data with sequences
patient_data = pd.read_pickle('Data/patient_data_sequences.pkl')
print("Loaded patient data with sequences.")

Load Code Mappings

In [ ]:
# Load code mappings
code_mappings = pd.read_csv('Data/code_mappings.csv')
code_to_id = dict(zip(code_mappings['UNIQUE_CODE'], code_mappings['CODE_ID']))
id_to_code = dict(zip(code_mappings['CODE_ID'], code_mappings['UNIQUE_CODE']))
num_codes = len(code_to_id)
print(f"Number of unique codes: {num_codes}")

Initialize Code Embeddings

In [ ]:
# Define embedding dimension
embedding_dim = 128  # Adjust as needed

# Initialize embeddings randomly
np.random.seed(42)
code_embeddings = np.random.normal(size=(num_codes, embedding_dim))

# Function to get embedding for a code ID
def get_code_embedding(code_id):
    return code_embeddings[code_id]


Aggregate Embeddings for Each Patient

In [ ]:
# Function to aggregate embeddings for a patient
def aggregate_patient_embeddings(visits):
    all_code_ids = [code_id for visit in visits for code_id in visit]
    if not all_code_ids:
        return np.zeros(embedding_dim)
    embeddings = np.array([get_code_embedding(code_id) for code_id in all_code_ids])
    mean_embedding = embeddings.mean(axis=0)
    return mean_embedding

# Aggregate embeddings for each patient
patient_embeddings = np.array([
    aggregate_patient_embeddings(row['SEQUENCE']) for _, row in patient_data.iterrows()
])

print("Aggregated embeddings for all patients.")


Prepare Demographic Features

In [ ]:
# Select demographic features
demographic_features = ['Id', 'AGE', 'DECEASED', 'GENDER', 'RACE', 'ETHNICITY',
                        'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE', 'INCOME']
demographics = patient_data[demographic_features]

# One-hot encode categorical variables
demographics = pd.get_dummies(demographics, columns=['GENDER', 'RACE', 'ETHNICITY'])

# Fill missing values
demographics.fillna(0, inplace=True)

print("Prepared demographic features with patient IDs.")


Combine Embeddings and Demographics

In [ ]:
# Convert demographics to NumPy array (excluding the 'Id' column)
demographics_array = demographics.drop(columns=['Id']).values

# Concatenate embeddings and demographics
X = np.hstack((patient_embeddings, demographics_array))

print(f"Final input shape: {X.shape}")


Standardise the Data

In [ ]:
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.joblib')
print("Data standardized and scaler saved.")


Train-Test Split
 
We need to split both the features and the patient IDs so that we can track which patients are in the training and validation sets.

In [ ]:
patient_ids = demographics['Id'].values

# Split into training and validation sets, including patient IDs
X_train, X_val, ids_train, ids_val = train_test_split(
    X_scaled, patient_ids, test_size=0.2, random_state=42)

# Clear unnecessary variables
del X, X_scaled, demographics_array
gc.collect()
# # Get patient IDs
# patient_ids = demographics['Id'].values

# # Split into training and validation sets, including patient IDs
# X_train, X_val, ids_train, ids_val = train_test_split(
#     X, patient_ids, test_size=0.2, random_state=42)

# # Standardize the data after splitting
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)

# # Save the scaler for future use
# joblib.dump(scaler, 'scaler.joblib')
# print("Data standardized and scaler saved.")

# # Clear unnecessary variables
# del X, patient_embeddings, demographics_array
# gc.collect()
# Get patient IDs



#### Define the Sampling Layer

In [4]:
# Sampling Layer
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    def get_config(self):
        config = super(Sampling, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)


#### Define the Variational Autoencoder (VAE) Model

In [5]:
# VAE Model with adjustable beta parameter
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, input_dim, beta=1.0, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.input_dim = input_dim
        self.beta = beta

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        
        # Reconstruction loss
        reconstruction_loss = tf.reduce_mean(
            tf.keras.losses.MeanSquaredError()(inputs, reconstructed)
        ) * self.input_dim
        
        # KL divergence loss
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        )
        
        # Total loss with beta parameter
        total_loss = reconstruction_loss + self.beta * kl_loss
        
        self.add_loss(total_loss)
        
        return reconstructed

    def get_config(self):
        config = super(VAE, self).get_config()
        config.update({
            'encoder': self.encoder.get_config(),
            'decoder': self.decoder.get_config(),
            'input_dim': self.input_dim,
            'beta': self.beta,
        })
        return config

    @classmethod
    def from_config(cls, config):
        encoder_config = config.pop('encoder')
        decoder_config = config.pop('decoder')

        encoder = tf.keras.Model.from_config(encoder_config)
        decoder = tf.keras.Model.from_config(decoder_config)

        return cls(encoder=encoder, decoder=decoder, **config)


#### Define the Model Builder Function for Hyperparameter Tuning

In [6]:
def build_vae(hp):
    input_dim = X_train.shape[1]
    
    # Hyperparameters
    num_layers = hp.Int('num_layers', 1, 3)
    units = hp.Int('units', 64, 512, step=64)
    activation = hp.Choice('activation', ['relu', 'tanh', 'selu', 'elu'])
    l2_reg = hp.Float('l2_reg', 1e-5, 1e-2, sampling='log')
    dropout_rate = hp.Float('dropout_rate', 0.1, 0.5, step=0.1)
    encoding_dim = hp.Int('encoding_dim', 8, 64, step=8)
    learning_rate = hp.Float('learning_rate', 1e-5, 1e-3, sampling='log')
    beta = hp.Float('beta', 1.0, 5.0, step=0.5)
    
    # Activation function
    activation_fn = activation
    
    # Encoder
    input_layer = layers.Input(shape=(input_dim,))
    x = input_layer
    for _ in range(num_layers):
        x = layers.Dense(
            units,
            activation=activation_fn,
            kernel_regularizer=tf.keras.regularizers.l2(l2_reg)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout_rate)(x)
    
    # Latent Space
    z_mean = layers.Dense(encoding_dim, name='z_mean')(x)
    z_log_var = layers.Dense(encoding_dim, name='z_log_var')(x)
    z = Sampling()([z_mean, z_log_var])
    
    # Encoder Model
    encoder = tf.keras.Model(inputs=input_layer, outputs=[z_mean, z_log_var, z], name='encoder')
    
    # Decoder
    latent_inputs = layers.Input(shape=(encoding_dim,))
    x = latent_inputs
    for _ in range(num_layers):
        x = layers.Dense(
            units,
            activation=activation_fn,
            kernel_regularizer=tf.keras.regularizers.l2(l2_reg)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout_rate)(x)
        
    outputs = layers.Dense(input_dim, activation='linear')(x)
    
    # Decoder Model
    decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name='decoder')
    
    # VAE Model
    vae = VAE(encoder, decoder, input_dim=input_dim, beta=beta)
    
    # Compile the model
    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    
    return vae


#### Set Up the Hyperparameter Tuner

In [1]:
# Set up the tuner
tuner = BayesianOptimization(
    build_vae,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='vae_tuning',
    project_name='vitai_vae_enhanced'
)

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


NameError: name 'BayesianOptimization' is not defined

#### Run the Hyperparameter Search

In [ ]:
# Run the hyperparameter search
tuner.search(
    X_train, X_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_val, X_val),
    callbacks=[early_stopping]
)

# Clear memory
gc.collect()
tf.keras.backend.clear_session()


#### Retrieve the Best Hyperparameters

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the optimal hyperparameters
print(f"""
The optimal number of layers is {best_hps.get('num_layers')} encoder and decoder layers.
The optimal number of units in each layer: {best_hps.get('units')}.
The optimal activation function is {best_hps.get('activation')}.
The optimal encoding dimension is {best_hps.get('encoding_dim')}.
The optimal dropout rate is {best_hps.get('dropout_rate')}.
The optimal L2 regularization is {best_hps.get('l2_reg')}.
The optimal beta value is {best_hps.get('beta')}.
The optimal learning rate is {best_hps.get('learning_rate')}.
""")


#### Build and Train the Best Model

In [ ]:
# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)

# Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

history = best_model.fit(
    X_train, X_train,
    epochs=200,
    batch_size=256,
    validation_data=(X_val, X_val),
    callbacks=[early_stopping, reduce_lr]
)

# Save the weights
best_model.save_weights('vae_weights.h5')

# Save the best hyperparameters
import pickle
with open('best_hyperparameters.pkl', 'wb') as f:
    pickle.dump(best_hps.values, f)

# Clear session to free memory
keras.backend.clear_session()
gc.collect()


#### Plot Training and Validation Loss

In [ ]:
# Plot the training and validation loss curves
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


#### Load the Best Model and Weights

In [11]:
# Load the best hyperparameters
with open('best_hyperparameters.pkl', 'rb') as f:
        best_hps_values = pickle.load(f)

# Reconstruct the hyperparameters object
best_hps = keras_tuner.HyperParameters()
best_hps.values = best_hps_values

# Rebuild the model architecture
best_model = build_vae(best_hps)

# Compile the model
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_hps.get('learning_rate')))

# Build the model by calling it on some data
dummy_input = tf.zeros((1, X_train.shape[1]))
best_model(dummy_input)

# Load the weights
best_model.load_weights('vae_weights.h5')


#### Compute Reconstruction Error

In [ ]:
# Compute reconstruction error for the entire training dataset
reconstructed = best_model.predict(X_train, batch_size=256)
reconstruction_errors = np.mean(np.square(X_train - reconstructed), axis=1)


#### Plot the Reconstruction Error Distribution

In [ ]:
# Plot the reconstruction error distribution
plt.figure(figsize=(10, 6))
plt.hist(reconstruction_errors, bins=50, color='blue', alpha=0.7, edgecolor='black')
plt.title('Reconstruction Error Distribution (Training Data)')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.show()


#### Obtain Latent Features from the Encoder

In [ ]:
# Use the encoder model directly from the best_model
encoder = best_model.encoder

# Obtain latent representation (using z_mean for better clustering)
z_mean, z_log_var, z = encoder.predict(X_train, batch_size=256)

# Use z_mean for clustering
latent_features = z_mean

# Create a DataFrame for latent features, including patient IDs
latent_dim = best_hps.get('encoding_dim')
latent_features_df = pd.DataFrame(
    data=latent_features,
    columns=[f'latent_{i}' for i in range(latent_dim)]
)
latent_features_df['Id'] = ids_train  # Add patient IDs

# Add reconstruction error to the latent features DataFrame
latent_features_df['reconstruction_error'] = reconstruction_errors

# Save the latent features
latent_features_df.to_csv('latent_features.csv', index=False)

# Clear memory
del z_mean, z_log_var, z
gc.collect()


#### Perform Clustering on Latent Features
Note: Some clustering algorithms can consume a significant amount of memory, especially with large datasets. To mitigate memory issues, we'll adjust our approach by using more memory-efficient algorithms like MiniBatchKMeans and sample the data for computing evaluation metrics.

In [ ]:
# Set the number of clusters to 10
n_clusters = 10

# Clustering algorithms to try
clustering_algorithms = {
    'MiniBatchKMeans': MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=1024),
    'GaussianMixture': GaussianMixture(n_components=n_clusters, random_state=42),
    'AgglomerativeClustering': AgglomerativeClustering(n_clusters=n_clusters),
}

best_algorithm = None
best_score = -1
best_labels = None

# Sample a subset of the data for computing evaluation metrics
sample_size = min(10000, latent_features.shape[0])  # Use up to 10,000 samples
if latent_features.shape[0] > sample_size:
    latent_features_sample, _, indices_sample, _ = train_test_split(
        latent_features, np.arange(latent_features.shape[0]),
        train_size=sample_size, random_state=42)
else:
    latent_features_sample = latent_features
    indices_sample = np.arange(latent_features.shape[0])

# Perform clustering
for name, algorithm in clustering_algorithms.items():
    print(f"\nTesting {name}...")
    try:
        cluster_labels = algorithm.fit_predict(latent_features)
        
        # Evaluate clustering
        cluster_labels_sample = cluster_labels[indices_sample]
        silhouette_avg = silhouette_score(latent_features_sample, cluster_labels_sample)
        print(f"{name} Silhouette Score: {silhouette_avg:.4f}")
        
        # Store the best algorithm based on Silhouette Score
        if silhouette_avg > best_score:
            best_score = silhouette_avg
            best_algorithm = name
            best_labels = cluster_labels
    except Exception as e:
        print(f"An error occurred with {name}: {e}")
        continue

print(f"\nBest algorithm: {best_algorithm} with Silhouette Score of {best_score:.4f}")

# Update latent_features_df with the best cluster labels
latent_features_df['cluster'] = best_labels

# Save the updated latent features
latent_features_df.to_csv('latent_features_with_clusters.csv', index=False)

# Clear memory
gc.collect()


#### Visualize Clusters Using t-SNE

In [ ]:
# Use t-SNE for visualization
print("Performing t-SNE...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_jobs=-1)
latent_2d = tsne.fit_transform(latent_features_sample)

# Add to DataFrame
latent_features_df_sample = latent_features_df.iloc[indices_sample].copy()
latent_features_df_sample['tsne_1'] = latent_2d[:, 0]
latent_features_df_sample['tsne_2'] = latent_2d[:, 1]

# Plot t-SNE
plt.figure(figsize=(10, 6))
sns.scatterplot(x='tsne_1', y='tsne_2', hue='cluster', data=latent_features_df_sample, palette='viridis', legend='full')
plt.title('Latent Space Visualization with t-SNE')
plt.show()


#### Visualize Clusters Using UMAP

In [ ]:
# Use UMAP for visualization
print("Performing UMAP...")
reducer = umap.UMAP(random_state=42)
latent_2d_umap = reducer.fit_transform(latent_features_sample)

# Add to DataFrame
latent_features_df_sample['umap_1'] = latent_2d_umap[:, 0]
latent_features_df_sample['umap_2'] = latent_2d_umap[:, 1]

# Plot UMAP
plt.figure(figsize=(10, 6))
sns.scatterplot(x='umap_1', y='umap_2', hue='cluster', data=latent_features_df_sample, palette='viridis', legend='full')
plt.title('Latent Space Visualization with UMAP')
plt.show()


#### Map Clusters to Severity Scores

In [18]:
# Map clusters to severity scores
cluster_severity = {cluster: index for index, cluster in enumerate(sorted(latent_features_df['cluster'].unique()))}
latent_features_df['severity_index'] = latent_features_df['cluster'].map(cluster_severity)

# Optionally, scale severity index to 0-10 range
scaler_severity = MinMaxScaler(feature_range=(0, 10))
latent_features_df['severity_index_scaled'] = scaler_severity.fit_transform(latent_features_df[['severity_index']])


Merge Latent Features with Demographics

In [ ]:
# Merge latent features with demographics using patient IDs
final_df = latent_features_df.merge(
    demographics, on='Id', how='left'
)

# Save the final DataFrame to a CSV file
final_df.to_csv('patient_clusters.csv', index=False)

print("Final DataFrame with patient IDs and clusters saved to 'patient_clusters.csv'.")


#### Analyze Clusters and Visualize Key Features

Combine Original Data with Cluster Labels

In [ ]:
# Create embedding column names
embedding_columns = [f'embedding_{i}' for i in range(patient_embeddings.shape[1])]

# Get demographic column names (excluding 'Id')
demographics_columns = demographics.drop(columns=['Id']).columns.tolist()

# Combine embedding and demographic column names
feature_columns = embedding_columns + demographics_columns

# Create DataFrame with the correct column names
analysis_df = pd.DataFrame(X_train, columns=feature_columns)

# Ensure that the indices align between analysis_df and latent_features_df
analysis_df.reset_index(drop=True, inplace=True)
latent_features_df.reset_index(drop=True, inplace=True)

# Add cluster labels and other information
analysis_df['cluster'] = latent_features_df['cluster'].values
analysis_df['severity_index'] = latent_features_df['severity_index_scaled'].values
analysis_df['reconstruction_error'] = latent_features_df['reconstruction_error'].values

# Save analysis DataFrame
analysis_df.to_csv("analysis.csv")


Compute Summary Statistics

In [ ]:
# Group by cluster and compute summary statistics
cluster_summary = analysis_df.groupby('cluster').mean()
print("\nCluster Summary Statistics:")
display(cluster_summary)


#### Visualize Reconstruction Error by Cluster

In [ ]:
# Visualize reconstruction error by cluster
plt.figure(figsize=(10, 6))
sns.boxplot(x='cluster', y='reconstruction_error', data=analysis_df)
plt.title('Reconstruction Error by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Reconstruction Error')
plt.show()


#### Visualize Key Features by Cluster

In [ ]:
# Fix the error by excluding 'Id' from the features
key_features = demographics.drop(columns=['Id']).columns.tolist()  # Exclude 'Id'

# Merge analysis_df with demographics for plotting
'''
Fix the error by excluding 'Id' from the features
We ensure only valid columns are included in the plotting function
Additionally, we merge the analysis_df with demographics to include the necessary features for plotting
'''
plot_df = analysis_df.merge(demographics[['Id'] + key_features], left_index=True, right_index=True)

# Limit to first 10 features for analysis
for feature in key_features[:10]:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='cluster', y=feature, data=plot_df)
    plt.title(f'Distribution of {feature} by Cluster')
    plt.xlabel('Cluster')
    plt.ylabel(feature)
    plt.show()


#### Clear Variables to Free Memory

In [ ]:
# Clear variables to free memory
del X_train
del X_val
del latent_features
del latent_2d
del latent_2d_umap
del encoder
del best_model
gc.collect()

print("Analysis complete.")
